### Fine-tuning pre-trained BERT for Sequence Classification model
#### Emotion labels that were used initially-
['serenity', 'joy', 'disgust', 'anxious', 'optimism', 'vigilance', 'sad', 'fear']

### Observations from implementation in emotion_analysis.ipynb -
#### Model scores were bad mostly due to less annotated data (emotions were added only for subsets of social and media groups' repsonses).
#### Training loss was monitored at steps and it was fluctuating up and down.
#### Validation loss was decreasing in each eval_step but it was sometimes falling below the training loss.
#### Validation loss was not showing an upward trajectory even if the number of epochs was increased, potentially indicating that overfitting wasn't happening so far, which could have been the case as there were a lot of features with all bert layers unfrozen and small dataset size.

### Things tried for improving the training loss fluctuation-
##### 1] Learning rates - 2e-5, default LR as part of Training Class args (5e-5), 0.0001, 0.001, 0.1
##### 2] Training+Validation dataset size tried - 50, 100, 200
##### 3] Using BERT instead of DistilBERT
##### 4] Changing the max_length for tokens - 512, 250, 200, 100, 80
##### 5] Freezing all layers to improve memory requirements but due to training loss appearing to oscillate, unfreezing all layers
##### and also freezing only the embeddings and initial trasnformer layers (upto 3) was tried
##### 6] experimenting by adding/substracting the following pipeline components for this specific task - stop word removal, lemmatizer, punctuations
##### 7] changing the number of epochs - 3, 5, 1, 10, 15, 30
##### 8] with limited data, increasing the training set size and very small validation  set size was also tried

##### Finally, to check if less training examples given 8 labels was causing an issue,  we reduced the number of labels- 
##### optimism was mapped to serenity
##### responses tagged under vigilance, sadness and fear all involved some level of anxiety and so all 3 were mapped to anxiety

#### Emotion labels that were used finally
['serenity', 'joy', 'disgust', 'anxious']

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn_crfsuite.metrics import flat_classification_report
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback
from transformers import AdamW
import torch.optim as optim

c:\users\vhpld\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\vhpld\anaconda3\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
c:\users\vhpld\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\vhpld\anaconda3\lib\site-packages\numpy\.libs\libopenblas.wcdjnk7yvmpzq2me2zzhjjrj3jikndb7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
#pip install contractions
#import contractions
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import regex as re

https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vhpld\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vhpld\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
def case_handling(df,columns=[]):
    
    for col in columns:
        df[col] = df[col].str.lower() 
        
    return df       

In [4]:
def remove_punctuations(df,columns=[]):
    
    for col in columns:
        df[col] = df[col].apply(lambda text: re.sub(r'[^\w\s]', '', text))
        
    return df

In [5]:
def remove_stopwords(df, columns=[]):
    
    stop_words = set(stopwords.words('english'))
    
    def remove_sw(text):
        txt_output = " ".join([word for word in str(text).split() if word not in stop_words])
        return txt_output
    
    for col in columns:
        df[col] = df[col].apply(lambda text: remove_sw(text))
    
    return df

In [6]:
def lemmatize_words(df, columns=[]):
    
    lemmatizer = WordNetLemmatizer()
    
    def lemmatize(text):
        text_output = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
        return text_output
    
    for col in columns:
        df[col] = df[col].apply(lambda text: lemmatize(text))
        
    return df

In [7]:
def remove_extra_spaces(df,columns=[]):
    
    for col in columns:
        df[col] = df[col].apply(lambda text: re.sub(' +', ' ', text))
        
    return df 

In [15]:
def data_preprocessing(df, columns=[]):
    
    #df = expand_contraction(df,columns)
    df = case_handling(df,columns) 
    df = remove_punctuations(df,columns)
    #df = remove_words_dgits(df,columns)  
    df = remove_stopwords(df,columns) 
    df = lemmatize_words(df, columns)
    df = remove_extra_spaces(df,columns) 
    
    return df

In [16]:
data = pd.read_csv("focus_groups_convos_emotion_analysis.csv", encoding="ISO-8859-1")

In [84]:
# response before pre-processing of text
data.iloc[2]['parent_answer']

"I have a similar experience to Parent 3. My high schooler had a lot of his homework that he had to type and word process... I don't know if that's the right word, into Docs, but it wasn't a huge amount. It wasn't doing research, it wasn't hours and hours. My elementary school guy, he didn't have any homework on the computer at all. And my middle schooler had very little, like type this one thing or something like that. It was very little on the computer. And for myself, in terms of my family, I lock the video games so that there was very little access to video games during the week."

In [17]:
columns=['parent_answer']
data =  data_preprocessing(data, columns)

In [18]:
data.head()

,focus_group_subtype,focus_group_subtype_id,doc_no_within_subtype,question_id,question_text,parent_num,parent_answer,emotion
0,gaming_group,1,1,1,So just starting to think through some of the ...,2,oh okay well didnt use much mean teacher would...,serenity
1,gaming_group,1,1,1,So just starting to think through some of the ...,3,would say thing daughter fifth grade would cou...,serenity
2,gaming_group,1,1,1,So just starting to think through some of the ...,5,similar experience parent 3 high schooler lot ...,serenity
3,gaming_group,1,1,1,So just starting to think through some of the ...,5,weekend hour something like wasnt much youtube...,joy
4,gaming_group,1,1,1,So just starting to think through some of the ...,4,go kid probably sound like online far rest dau...,serenity


In [19]:
# response after pre-processing of text
data['parent_answer'].max()

'youtube bottomless hole cant even begin bad transition looking google classroom google meet here youtube easy always still job thing cant sit top accessibility technology type content like youtube thats beneficial kid thats problematic say least'

In [20]:
data['emotion'].value_counts()

anxious     101
serenity     71
disgust      44
joy          13
Name: emotion, dtype: int64

In [21]:
emotions = list(data['emotion'].unique())
label_to_index = {emotion : index for index, emotion in enumerate(emotions)}
label_to_index

{'serenity': 0, 'joy': 1, 'disgust': 2, 'anxious': 3}

In [22]:
data['emotion_label_index'] = data.apply(lambda x : label_to_index[x['emotion']], axis=1)

In [23]:
data.head()

,focus_group_subtype,focus_group_subtype_id,doc_no_within_subtype,question_id,question_text,parent_num,parent_answer,emotion,emotion_label_index
0,gaming_group,1,1,1,So just starting to think through some of the ...,2,oh okay well didnt use much mean teacher would...,serenity,0
1,gaming_group,1,1,1,So just starting to think through some of the ...,3,would say thing daughter fifth grade would cou...,serenity,0
2,gaming_group,1,1,1,So just starting to think through some of the ...,5,similar experience parent 3 high schooler lot ...,serenity,0
3,gaming_group,1,1,1,So just starting to think through some of the ...,5,weekend hour something like wasnt much youtube...,joy,1
4,gaming_group,1,1,1,So just starting to think through some of the ...,4,go kid probably sound like online far rest dau...,serenity,0


In [24]:
index_to_label = {index: label for label, index in label_to_index.items()}
index_to_label

{0: 'serenity', 1: 'joy', 2: 'disgust', 3: 'anxious'}

In [217]:
""" tried but not used finally
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_index))
"""

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

In [25]:
learning_rate = 2e-5

In [128]:
### freezing all layers except the classification head when distilbert was tried
for param in model.distilbert.parameters():
    param.requires_grad = False

optimizer = optim.Adam([{'params' : model.classifier.parameters()}], lr=1)


"""
Following was not used -
changing the optimizer
differential learning rates

optimizer = AdamW(filter(lambda p : p.requires_grad, model.parameters()), lr = learning_rate)
optim.SGD([
                {'params': model.base.parameters()},
                {'params': model.classifier.parameters(), 'lr': 1e-3}
            ], lr=1e-2, momentum=0.9)
"""

"\noptimizer = AdamW(filter(lambda p : p.requires_grad, model.parameters()), lr = learning_rate)\noptim.SGD([\n                {'params': model.base.parameters()},\n                {'params': model.classifier.parameters(), 'lr': 1e-3}\n            ], lr=1e-2, momentum=0.9)\n"

In [16]:
### freezing only the initial pre-trained distilbert model's layers
modules = [model.distilbert.embeddings, model.distilbert.transformer.layer[:2]]
for module in modules:
    for param in module.parameters():
        param.require_grad = False

In [286]:
optimizer??

In [14]:
model.distilbert??

In [64]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [26]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_to_index))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [27]:
X = list(data["parent_answer"])
y = list(data["emotion_label_index"])

In [188]:
X

["oh okay. well, use much. mean, teacher would assign reading math assignment ready time time homework. yeah, it. would monitor much time roblox thing entertain themselves, easier control back go much. mean, that's was.",
 'would say thing. daughter fifth grade, would couple newsela projects, half hour homework. playground, neighborhood friends, school projects, school swimming. little bit tablet weekends, internet access youtube life, roblox, gaming. so, different world.',
 "similar experience parent 3. high schooler lot homework type word process... know that's right word, docs, huge amount. research, hour hours. elementary school guy, homework computer all. middle schooler little, like type one thing something like that. little computer. myself, term family, lock video game little access video game week.",
 "go. kid probably more... sound like online than, far, rest them. daughter high school definitely almost homework... assigned school, definitely homework, feel like, good amount.

In [189]:
y

[0,
 0,
 0,
 0,
 1,
 1,
 2,
 1,
 2,
 0,
 2,
 2,
 1,
 0,
 0,
 1,
 0,
 0,
 3,
 1,
 2,
 3,
 2,
 2,
 2,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 3,
 3,
 3,
 3,
 1,
 3,
 2,
 2,
 3,
 2,
 1,
 3,
 3,
 1,
 2,
 1,
 0,
 3,
 2,
 2,
 3,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 3,
 3,
 3,
 2,
 3,
 1,
 3,
 0,
 2,
 0,
 1,
 0,
 2,
 0,
 2]

In [35]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

In [27]:
train_test_split??

In [36]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=80)

In [39]:
type(X_train_tokenized)

transformers.tokenization_utils_base.BatchEncoding

In [37]:
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=80)

In [38]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels
        #print(labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        #print(item)
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [39]:
train_dataset = Dataset(X_train_tokenized, y_train)

In [47]:
print(train_dataset[3])

{'input_ids': tensor([  101,  3398,  1012,  2714,  6687,  1019,  6687,  1017,  1010,  2812,
         1010,  4845,  2052, 17781,  3042,  1012, 18546,  8654, 25249,  3047,
         1012,  2288, 18546, 17470,  2082,  1012,  2812,  1010,  1045,  1005,
         1049,  2551,  2188,  1012,  2093,  4845, 26536,  9354,  1012,  3034,
         4455,  1010,  2562,  4251,  1012,  2288, 25249,  2015,  2346, 23467,
         3047,  1010,  2008,  1005,  1055, 10639,  2814,  1012,  2253, 19448,
         1010,  4121,  2518,  2064,  1005,  1056,  1012,  1012,  1012,  2027,
         1005,  2128, 23042,  1012,  2448,  2188,  2082,  1010,  1000,  2073,
         1005,  1055, 25249,  1029,  2215,  2377,  1012,  1000,  2009,  1005,
         1055,  1010,  1045,  1005,  1049,  2145,  2551,  1012,  2507,  2210,
         2978,  1010,  2215,  2131,  2125,  1012,  2812,  1010,  2428,  1012,
         1012,  1012,  2052,  2377,  2208,  2030,  1012,  1012,  1012,  2082,
         1010, 10124,  2707,  1012,  2059,  1010, 

In [40]:
val_dataset = Dataset(X_val_tokenized, y_val)

In [41]:
def compute_metrics(p):
    
    pred, labels = p
    #print(pred)
    #print(pred.shape)
    pred = np.argmax(pred, axis=1)
    """
    labels = p.label_ids
    pred = p.predictions.argmax(-1)
    print(labels)
    print(pred)
    """
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average = 'weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average = 'weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average = 'weighted')
    #print(f"accuracy: {accuracy}, precision: {precision}, recall: {recall}, f1: {f1}")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [271]:
accuracy_score??

In [50]:
TrainingArguments??

In [42]:

args = TrainingArguments(
    output_dir="focus_groups_emotion_output",
    evaluation_strategy="steps",
    eval_steps=5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=15,
    num_train_epochs=10,
    seed=0,
    load_best_model_at_end=True,
    logging_steps=1,
    learning_rate = learning_rate
)
#learning_rate = learning_rate

In [198]:
args

TrainingArguments(output_dir=focus_groups_emotion_output, overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.STEPS, prediction_loss_only=False, per_device_train_batch_size=64, per_device_eval_batch_size=18, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs\Dec04_19-18-40_DESKTOP-RR0FD0F, logging_strategy=IntervalStrategy.STEPS, logging_first_step=False, logging_steps=1, save_strategy=IntervalStrategy.STEPS, save_steps=500, save_total_limit=None, no_cuda=False, seed=0, fp16=False, fp16_opt_level=O1, fp16_backend=auto, fp16_full_eval=False, local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False, debug=[], dataloader_drop_last=False, eval_steps=10, dataloader_num_workers=0, past_i

In [43]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    
)

In [275]:
Trainer.__init__??

In [290]:
TrainingArguments??

In [44]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
5,1.272500,1.159274,0.571429,0.441964,0.571429,0.473046
10,1.234700,1.092833,0.571429,0.441964,0.571429,0.473046
15,1.102700,1.065651,0.542857,0.345455,0.542857,0.422222
20,0.994200,1.079866,0.600000,0.495961,0.600000,0.529469
25,1.108500,1.073334,0.571429,0.471429,0.571429,0.509388
30,0.948200,1.084212,0.571429,0.471429,0.571429,0.509388


c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedM

TrainOutput(global_step=30, training_loss=1.1574009736378987, metrics={'train_runtime': 2648.5589, 'train_samples_per_second': 0.015, 'total_flos': 0, 'epoch': 7.5, 'init_mem_cpu_alloc_delta': 147456, 'init_mem_cpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1603510272, 'train_mem_cpu_peaked_delta': 3570049024})

In [ ]:
## earlier run when training loss was fluctuating 
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.107800,2.059592,0.250000,0.105114,0.250000,0.147431
2,2.187000,2.048275,0.318182,0.101240,0.318182,0.153605
3,2.077000,2.036327,0.318182,0.101240,0.318182,0.153605
4,2.150000,2.029730,0.318182,0.101240,0.318182,0.153605
5,2.066900,2.022511,0.318182,0.101240,0.318182,0.153605
6,1.986700,2.016541,0.318182,0.101240,0.318182,0.153605
7,2.031800,2.010084,0.318182,0.103594,0.318182,0.156300
8,2.078600,2.006244,0.318182,0.103594,0.318182,0.156300
9,2.023000,2.002145,0.295455,0.100887,0.295455,0.150413
10,1.852300,1.996485,0.295455,0.108852,0.295455,0.159091


[[-2.55430222e-01  5.23206079e-03  7.16976151e-02  1.95126578e-01
   2.06720665e-01 -8.07537064e-02 -4.61732745e-02  8.76906961e-02]
 [-2.38123000e-01  8.29945784e-03 -7.06706196e-03  2.18617037e-01
   2.30133817e-01 -1.01340972e-01 -8.22402388e-02 -1.80257857e-03]
 [-2.10962832e-01  1.43473549e-02  4.01571281e-02  2.37562120e-01
   2.16211408e-01 -1.07282706e-01 -3.60744633e-02  6.73384368e-02]
 [-2.52977580e-01  2.57163122e-03  1.14885233e-02  2.29033142e-01
   2.17481434e-01 -1.01586983e-01 -7.61057362e-02 -2.07156166e-02]
 [-2.10639536e-01  9.72980633e-05  2.04135142e-02  2.32257470e-01
   2.08822310e-01 -1.03895500e-01 -6.85415268e-02  4.99580950e-02]
 [-2.28587016e-01 -1.58852562e-02 -9.00641084e-04  2.69581854e-01
   1.85750589e-01 -8.09553564e-02 -7.40135238e-02  7.46334344e-03]
 [-2.39813611e-01 -1.24258548e-02  2.57445127e-03  2.42303193e-01
   1.94305018e-01 -1.27500877e-01 -6.96335807e-02  1.44356638e-02]
 [-2.29257271e-01 -4.02236693e-02 -5.09312041e-02  2.22777903e-01
   

c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 0.25, precision: 0.10511363636363637, recall: 0.25, f1: 0.1474308300395257
[[-2.31532976e-01 -1.65545736e-02  9.61091146e-02  1.94729924e-01
   1.82518870e-01 -9.84241217e-02 -3.80140543e-03  5.18569276e-02]
 [-2.14018732e-01 -1.72333065e-02  1.89514533e-02  2.16189221e-01
   2.04298645e-01 -1.20588452e-01 -3.70064117e-02 -3.97562161e-02]
 [-1.85813889e-01 -8.97854194e-03  6.37502968e-02  2.38005847e-01
   1.89622015e-01 -1.27683118e-01  8.23188201e-03  3.13110277e-02]
 [-2.27006495e-01 -2.28872467e-02  3.40416469e-02  2.25916103e-01
   1.89020172e-01 -1.24110982e-01 -3.14549990e-02 -5.86730987e-02]
 [-1.85245559e-01 -2.48228759e-02  4.60463911e-02  2.30051666e-01
   1.81226388e-01 -1.22390762e-01 -1.97695419e-02  1.32618621e-02]
 [-2.01604992e-01 -3.96379307e-02  2.38833539e-02  2.68069237e-01
   1.54532224e-01 -1.01031899e-01 -2.81405151e-02 -2.83430666e-02]
 [-2.15465575e-01 -3.66927385e-02  2.51892731e-02  2.40684330e-01
   1.68280110e-01 -1.47367477e-01 -2.75212638e-02 -

c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[ 0.1701029  -0.12869859  0.43581378  0.35237604 -0.05302569 -0.4149249
   0.14788772 -0.42574978]
 [ 0.19441018 -0.14089799  0.3857175   0.38613573 -0.04308029 -0.46517956
   0.13337392 -0.5323505 ]
 [ 0.23581836 -0.11218637  0.39112747  0.42505288 -0.04212994 -0.47075206
   0.17968155 -0.4478426 ]
 [ 0.23058856 -0.13873206  0.38316968  0.373416   -0.07268393 -0.46370676
   0.1253979  -0.54840755]
 [ 0.23895524 -0.13042323  0.39640582  0.3945656  -0.07841083 -0.4567799
   0.15680146 -0.44059208]
 [ 0.23197995 -0.13950586  0.37011623  0.45430902 -0.10868491 -0.4384696
   0.14788523 -0.47885942]
 [ 0.18331414 -0.14103708  0.37001136  0.3895989  -0.07966995 -0.45715445
   0.1137986  -0.47351056]
 [ 0.21049464 -0.18071051  0.2884139   0.37739837 -0.05168004 -0.48535728
   0.20643686 -0.45967025]
 [ 0.204211   -0.14582975  0.31778294  0.3626092  -0.07784801 -0.4667578
   0.17072462 -0.47451532]
 [ 0.22711477 -0.12419406  0.4341843   0.39301088 -0.04456954 -0.45620525
   0.14079604 -0.4784

c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[ 0.17939964 -0.13435483  0.4521311   0.3596064  -0.05612822 -0.42465118
   0.14289804 -0.44083667]
 [ 0.20481053 -0.14675087  0.40184847  0.39417794 -0.04631893 -0.47638908
   0.12822188 -0.5487033 ]
 [ 0.24640788 -0.1175879   0.4061222   0.43298316 -0.04446376 -0.48119855
   0.17521471 -0.46378523]
 [ 0.24222031 -0.14464006  0.39957845  0.38096362 -0.07662984 -0.47493696
   0.12031317 -0.56339204]
 [ 0.24956134 -0.13554943  0.4124986   0.40156    -0.08264989 -0.46786514
   0.1529988  -0.45639044]
 [ 0.2431571  -0.14472097  0.38646173  0.46248955 -0.11310992 -0.45016676
   0.14293161 -0.4937638 ]
 [ 0.19282153 -0.1453598   0.3855518   0.39604542 -0.08310805 -0.46741825
   0.10908917 -0.4890769 ]
 [ 0.22098225 -0.18610261  0.3036906   0.38421708 -0.05627968 -0.49522737
   0.20266104 -0.47449052]
 [ 0.21543375 -0.1513539   0.3329238   0.3686035  -0.08258102 -0.4769563
   0.16659398 -0.48901173]
 [ 0.2382538  -0.12944642  0.4507034   0.39997303 -0.04865025 -0.4667049
   0.1356172  -0.49

c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [115]:
trainer.train??

In [45]:
test_data = pd.read_csv("focus_groups_convos_emotion_analysis_test.csv")

In [46]:
test_data["emotion"].value_counts()

anxious     4
serenity    3
disgust     1
joy         1
Name: emotion, dtype: int64

In [47]:
emotions = list(test_data['emotion'].unique())
#TODO: new labels in test data handling
test_data['emotion_label_index'] = test_data.apply(lambda x : label_to_index[x['emotion']], axis=1)

In [48]:
columns=['parent_answer']
test_data =  data_preprocessing(test_data, columns)

In [49]:
X_test = list(test_data["parent_answer"])
y_test = list(test_data["emotion_label_index"])

In [34]:
X_test

['yeah similar experience eightyearold prepandemic still second grade use computer limited maybe week would type word week one option picked interaction computer course device play would go father house every weekend would exposed computer still busy busy week school getting school getting school thats trip',
 'hockey taekwondo busy even didnt seem like enough didnt seem like enough certainly lot busier get play mostly probably weekend weekend also pretty packed dont know time play lot course able control item earn didnt well dont get play tool could use incentive week thats window available right day long getting away computer project every day',
 'like bomb went figured figured workarounds like pandora box trying figure block phone doe phone parent coordinator calling principal like block youtube without putting timer thing driving crazy downhill finally figured son banging much he actually broken screen whatever control put could bang would whatever wanted month later exchange ipad 

In [206]:
y_test

[0, 2, 1, 0, 2, 2, 1, 0, 1, 2, 2, 1, 0, 3, 1, 0, 1, 1, 2, 2, 3]

In [50]:
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=80)

In [161]:
X_test_tokenized

{'input_ids': [[101, 3398, 1010, 2714, 3325, 2809, 1011, 2095, 1011, 2214, 1012, 3653, 1011, 6090, 3207, 7712, 2145, 2117, 3694, 2224, 3274, 3132, 1012, 2672, 2733, 2052, 2828, 1010, 2773, 2733, 2028, 5724, 3856, 2079, 1012, 8290, 3274, 1012, 2607, 5080, 2377, 2006, 1010, 2052, 2175, 2269, 1005, 1055, 2160, 2296, 5353, 2052, 6086, 7588, 1012, 2145, 5697, 1012, 5697, 2733, 1010, 2082, 1010, 2893, 2082, 1010, 2893, 2082, 1010, 2008, 1005, 1055, 4440, 2993, 1012, 102, 0, 0, 0, 0], [101, 2066, 5968, 2253, 2125, 1010, 6618, 2041, 1010, 6618, 2147, 24490, 2015, 2066, 19066, 1005, 1055, 3482, 1012, 2667, 3275, 3796, 1012, 1012, 1012, 3042, 18629, 1010, 3042, 6687, 10669, 1010, 4214, 4054, 1010, 2066, 1010, 1000, 2129, 3796, 7858, 2302, 1012, 1012, 1012, 1000, 5128, 25309, 2477, 1010, 4439, 4689, 1010, 19448, 2045, 1012, 2633, 6618, 2365, 1010, 22255, 2172, 2002, 1005, 1055, 2941, 3714, 3898, 3649, 2491, 2404, 1999, 1010, 2071, 9748, 2052, 3649, 2359, 1012, 102], [101, 2036, 6134, 1012, 2569, 

In [51]:
test_dataset = Dataset(X_test_tokenized, y_test)

In [209]:
test_dataset[0]

{'input_ids': tensor([ 101, 3398, 1010, 2714, 3325, 2809, 1011, 2095, 1011, 2214, 1012, 3653,
         1011, 6090, 3207, 7712, 2145, 2117, 3694, 2224, 3274, 3132, 1012, 2672,
         2733, 2052, 2828, 1010, 2773, 2733, 2028, 5724, 3856, 2079, 1012, 8290,
         3274, 1012, 2607, 5080, 2377, 2006, 1010, 2052, 2175, 2269, 1005, 1055,
         2160, 2296, 5353, 2052, 6086, 7588, 1012, 2145, 5697, 1012, 5697, 2733,
         1010, 2082, 1010, 2893, 2082, 1010, 2893, 2082, 1010, 2008, 1005, 1055,
         4440, 2993, 1012,  102,    0,    0,    0,    0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 0]),
 'labels': tensor(0)}

In [52]:
test_trainer = Trainer(model)

In [132]:
test_trainer.predict??

In [53]:
raw_pred, label_ids, metrics = test_trainer.predict(test_dataset)

In [39]:
raw_pred

array([[ 0.06766115, -0.3253832 , -0.06493668,  0.35471714, -0.15618443,
        -0.11512066, -0.16246825, -0.06664449],
       [ 0.0916235 , -0.35716227, -0.06221877,  0.41920525, -0.1804009 ,
        -0.17619711, -0.06709153,  0.04494047],
       [ 0.04280779, -0.37059927, -0.04707661,  0.34497947, -0.17658278,
        -0.14099593, -0.0756123 ,  0.05449395],
       [-0.1484484 , -0.6329201 , -0.34090024,  0.02485409,  0.0903853 ,
        -0.28749904, -0.05994736,  0.20962328],
       [ 0.03568046, -0.19395709,  0.02585419,  0.43622717, -0.05606452,
        -0.24524978, -0.17073739,  0.12783825],
       [ 0.00079341, -0.51856947, -0.12783314,  0.2570194 , -0.02892824,
        -0.38981146,  0.03351265,  0.04641962],
       [-0.07587789, -0.75548315, -0.28536007,  0.22248712,  0.08766535,
        -0.57228875,  0.26338446,  0.34008944],
       [-0.01004304, -0.46937793, -0.19732882,  0.2263671 , -0.07728738,
        -0.17946137, -0.05250129,  0.01034988],
       [-0.053477  , -0.5714641 

In [54]:
y_pred = np.argmax(raw_pred, axis=1)

In [55]:
y_pred = list(y_pred)
y_pred

[3, 3, 3, 3, 3, 0, 3, 3, 0]

In [56]:
label_ids = list(label_ids)
label_ids

[3, 0, 2, 1, 3, 0, 0, 3, 3]

In [57]:
pred = [ index_to_label[p] for p in y_pred]
print(pred)
true = [ index_to_label[label_id] for label_id in label_ids]
print(true)
report = classification_report(y_true=true, y_pred=pred)
print(report)

['anxious', 'anxious', 'anxious', 'anxious', 'anxious', 'serenity', 'anxious', 'anxious', 'serenity']
['anxious', 'serenity', 'disgust', 'joy', 'anxious', 'serenity', 'serenity', 'anxious', 'anxious']
              precision    recall  f1-score   support

     anxious       0.43      0.75      0.55         4
     disgust       0.00      0.00      0.00         1
         joy       0.00      0.00      0.00         1
    serenity       0.50      0.33      0.40         3

    accuracy                           0.44         9
   macro avg       0.23      0.27      0.24         9
weighted avg       0.36      0.44      0.38         9



c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
## saving model once satisfactory scores are obtained
model.save_pretrained('emotion_analysis_output')
### NOTE: unable to upload to GIT the pytorch_model.bin serialized file as it is 427 MB

In [73]:
# loading model fine-tuned for emotion analysis
finetuned_model = BertForSequenceClassification.from_pretrained(r'C:\Users\vhpld\Desktop\emotion_analysis_output')

In [74]:
#testing the results match with the above model
test_trainer1 = Trainer(finetuned_model)

In [75]:
raw_pred, label_ids, metrics = test_trainer1.predict(test_dataset)

In [76]:
y_pred1 = list(np.argmax(raw_pred, axis=1))

In [77]:
label_ids1 = list(label_ids)

In [78]:
pred=['A', 'B']
true=['A', 'B']
pred = [ index_to_label[p] for p in y_pred1]
print(pred)
true = [ index_to_label[label_id] for label_id in label_ids1]
print(true)
report = classification_report(y_true=true, y_pred=pred)
print(report)

['anxious', 'anxious', 'anxious', 'anxious', 'anxious', 'serenity', 'anxious', 'anxious', 'serenity']
['anxious', 'serenity', 'disgust', 'joy', 'anxious', 'serenity', 'serenity', 'anxious', 'anxious']
              precision    recall  f1-score   support

     anxious       0.43      0.75      0.55         4
     disgust       0.00      0.00      0.00         1
         joy       0.00      0.00      0.00         1
    serenity       0.50      0.33      0.40         3

    accuracy                           0.44         9
   macro avg       0.23      0.27      0.24         9
weighted avg       0.36      0.44      0.38         9



c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\vhpld\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Conclusion -
##### With limited data, hyerparameter tuning with 8 classes was not leading to good scores and training loss was fluctuating
##### Labels were reduced to 4 from 8
##### 4 labels were chosed in a such a way so as to capture varying degrees of positive (serenity, joy) and negative emotions (anxious, disgust)
##### Reduction in labels finally led to the training loss fluctuation pattern to disappear and more labels were detected in the output as opposed to only single label being detected when training loss was fluctuating and not declining even after running too many epochs (assuming the batch size was causing the training loss to appear to not converge)

#### Next step-
##### Annotating the lowPIU and media groups' parent level responses with emotions
##### Improvement observed on very small test dataset size and this needs to be increased